# Multi-channel sliding window feature generation

In [8]:
import os
import numpy as np
import pandas as pd
import itertools
import glob

# Load dataset
data = 'B' # ****** specify dataset (A / B / C / D) ******
dataset = pd.read_csv(f'../data/data_{data}.csv')
dataset_name='dataSet'+data

min_window = 2 # ****** specify minimum window size ******
max_window = 2 # ****** specify maximum window size ******

In [9]:
# Generate features
all_windows=[]
for window_size in range(min_window,max_window+1):
    step=1
    ranges = [range(2)]*window_size
    windows=[]
    df_all = []
    for xs in itertools.product(*ranges):
        windows.append([*xs])
    all_windows.append(windows)
    print(f'generating features with window size {window_size}...')
    for window in windows:
        sliding_activation=[]
        for line_n in range(len(dataset)):
            line = list(dataset.iloc[line_n,:])
            region_activation = []
            for start in range(32-window_size):
                region = list(dataset.iloc[:,1:-1].iloc[line_n,start:start+window_size])
                if region == window:
                    region_activation.append(1)
                else:
                    region_activation.append(0)
                start+=step
            sliding_activation.append(region_activation)
        pd.DataFrame(sliding_activation).to_csv(f'./window_size_{window_size}_window_{window}_dataset_{data}.csv',index=False)

generating features with window size 2...


In [10]:
# Combine features
for i, window_size in enumerate(range(min_window,max_window+1)):
    windows = all_windows[i]
    df_all = []
    for col in range(32-window_size):
        all_column=[]
        for window in windows:
            column = list(pd.read_csv(f'./window_size_{window_size}_window_{window}_dataset_{data}.csv').iloc[:,col])
            all_column.append(column)
        df_all.append(list(zip(*all_column)))
    
    # Output features
    transpose_m = np.transpose(np.array(df_all),(1,0,2))
    df_save = np.reshape(transpose_m,(len(transpose_m), -1))
    np.savetxt(f"./sliding_window_{window_size}_dataset_{data}.txt",df_save)

    # Remove all redundant features
    for filename in glob.glob('window_size_*'):
        os.remove(filename)